In [38]:
from itertools import combinations
from collections import defaultdict, deque

def compute_H_A_and_H_B_with_root(tree, graph, n, max_degree, root, A, B):
    """
    Compute the combined H_A and H_B energy terms for a given tree configuration,
    considering a specific root.

    Args:
        tree: list of edges included in the tree.
        graph: dict, adjacency list of the graph.
        n: int, number of nodes in the graph.
        max_degree: int, maximum degree constraint for any vertex.
        root: int, node to be treated as the root.
        A: float, penalty weight for constraints.
        B: float, weight for the cost term.

    Returns:
        H: float, the combined energy of the configuration.
    """
    # Initialize depth and degree variables
    depth = defaultdict(lambda: float('inf'))  # Depth of each node, inf for unvisited
    degrees = defaultdict(int)  # Degree of each node
    visited = set()

    # Set root depth to 0
    depth[root] = 0

    # Simulate depth assignment using BFS
    queue = deque([root])
    while queue:
        node = queue.popleft()
        visited.add(node)
        for edge in tree:
            u, v = edge
            if u == node and v not in visited:
                depth[v] = min(depth[v], depth[u] + 1)
                queue.append(v)
            elif v == node and u not in visited:
                depth[u] = min(depth[u], depth[v] + 1)
                queue.append(u)

    # Calculate degrees
    for edge in tree:
        u, v = edge
        degrees[u] += 1
        degrees[v] += 1

    # Constraint terms
    single_root = 0  # Exactly one root is explicitly set
    unique_depth = A * sum(1 if depth[v] == float('inf') else 0 for v in graph)
    degree_constraint = A * sum(max(0, d - max_degree) for d in degrees.values())
    connectivity = A * (n - len(visited)) ** 2  # All nodes must be reachable

    # H_A energy
    H_A = single_root + unique_depth + degree_constraint + connectivity

    # H_B energy
    H_B = B * len(tree)  # Each included edge contributes to H_B with uniform cost

    return H_A + H_B


def generate_all_spanning_trees_with_H_and_roots(graph, max_degree, A=10, B=1):
    """
    Generate all spanning trees, considering all nodes as potential roots, and compute
    H = H_A + H_B to find the minimal-energy tree.

    Args:
        graph: dict, adjacency list of the graph {node: [neighbors]}.
        max_degree: int, maximum degree constraint for any vertex.
        A: float, penalty weight for constraints.
        B: float, weight for the cost term.

    Returns:
        min_tree: list of edges in the optimal spanning tree.
        min_H: float, the minimal energy (H).
        best_root: int, the root of the minimal-energy tree.
    """
    nodes = list(graph.keys())
    edges = [(u, v) for u in graph for v in graph[u] if u < v]
    n = len(nodes)

    min_H = float('inf')
    min_tree = None
    best_root = None

    # Enumerate all subsets of edges and all possible roots
    for edge_subset in combinations(edges, n - 1):
        for root in nodes:
            H = compute_H_A_and_H_B_with_root(edge_subset, graph, n, max_degree, root, A, B)
            if H < min_H:
                min_H = H
                min_tree = edge_subset
                best_root = root

    return min_tree, min_H, best_root

# Example Usage
if __name__ == "__main__":
    # Define the graph as an adjacency list
    graph9 = {
        0: [1, 2],
        1: [0, 2, 3],
        2: [0, 1, 3],
        3: [1, 2],
    }
    graph8 = {
        0: [1],  # Node 0 is connected to Node 1
        1: [0]   # Node 1 is connected to Node 0
    }
    graph7 = {
        0: [1],       # Node A is connected to Node B
        1: [0, 2],    # Node B is connected to Node A and Node C
        2: [1, 4],    # Node C is connected to Node B and Node E
        3: [4],       # Node D is connected to Node E
        4: [2, 3]     # Node E is connected to Node C and Node D
    }
    graph6 = {
        0: [1],       # Node D is connected to Node E
        1: [0],       # Node E is connected to Node D
        2: [3, 4],    # Node A is connected to Node B and Node C
        3: [2, 4],    # Node B is connected to Node A and Node C
        4: [2, 3]     # Node C is connected to Node A and Node B
    }
    graph5 = {
        0: [1, 2],    # Node A is connected to Node B and Node C
        1: [0, 2],    # Node B is connected to Node A and Node C
        2: [0, 1, 3], # Node C is connected to Node A, Node B, and Node D
        3: [2, 4],    # Node D is connected to Node C and Node E
        4: [3]        # Node E is connected to Node D
    }
    max_degree = 2
    A = 10  # Weight for constraints
    B = 1   # Weight for cost term

    # Find the optimal spanning tree
    min_tree, min_H, best_root = generate_all_spanning_trees_with_H_and_roots(graph6, max_degree, A, B)

    # Print the results
    if min_tree:
        print("Optimal Spanning Tree:", min_tree)
        print("Minimum Energy (H = H_A + H_B):", min_H)
        print("Best Root:", best_root)
    else:
        print("No valid spanning tree found.")


Optimal Spanning Tree: ((0, 1), (2, 3), (2, 4), (3, 4))
Minimum Energy (H = H_A + H_B): 64
Best Root: 2
